In [ ]:
import unicodedata
import string
import re
import random
import time
import datetime
import math
import socket
hostname = socket.gethostname()

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence#, masked_cross_entropy


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline
import  math
import  torch
import  random
from  torch  import  nn
from  torch.autograd  import Variable
from torch.optim import Adam
import torch.nn.functional as F
import torchtext
from torchtext import datasets
from torchtext import data
from torchtext.datasets import Multi30k, LanguageModelingDataset
from torchtext.data import Field, BucketIterator, Iterator
from torch.nn.utils import clip_grad_norm
import spacy

In [ ]:
USE_CUDA = True

In [ ]:
import pandas as pd

In [ ]:
import spacy

class tokenize(object):
    
    def __init__(self, lang):
        self.nlp = spacy.load(lang)
            
    def tokenizer(self, sentence):
        sentence = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        return [tok.text for tok in self.nlp.tokenizer(sentence) if tok.text != " "]

In [ ]:
tokenize_fr = tokenize('fr')
tokenize_en = tokenize('en')

In [ ]:
FR_TEXT = data.Field(lower=True, tokenize=tokenize_fr.tokenizer, init_token='<sos>', eos_token='<eos>')
EN_TEXT = data.Field(lower=True, tokenize=tokenize_en.tokenizer, init_token='<sos>', eos_token='<eos>')

In [ ]:
datafields = [("French", FR_TEXT), ("English", EN_TEXT)]
tr, v = data.TabularDataset.splits(path='./', train="train.csv", validation="val.csv", format = "csv", fields=datafields)

In [ ]:
FR_TEXT.build_vocab(tr, v)
EN_TEXT.build_vocab(tr, v)

In [ ]:
from torchtext.data import TabularDataset

datafields = [("id", None), ("French", FR_TEXT), ("English", EN_TEXT)]
trn = TabularDataset("French.csv", format='csv', fields=datafields)

FR_TEXT.build_vocab(trn)
EN_TEXT.build_vocab(trn)

In [ ]:
batch_size = 50
md = BucketIterator(tr, batch_size=batch_size, sort_key=lambda x: len(x.English), shuffle=True)

In [ ]:
def random_batch(batch_size):
    
    batch = next(iter(md))
      
    input_batch = batch.English[1:].cuda()
    target_batch = batch.French[1:].cuda()
    
    target_lengths = []
    input_lengths = []
    
    input_tp = input_batch.transpose(0,1).data.cpu().numpy()
    target_tp = target_batch.transpose(0,1).data.cpu().numpy()

    for i in range(batch_size):
        target_lengths.append(int(np.where(target_tp[i] == 3)[0][0] + 1))
        input_lengths.append(int(np.where(input_tp[i] == 3)[0][0] + 1))
    
   
    p = list(np.array((input_batch.data.transpose(0,1) == 3).nonzero()[:,1]))
    return input_batch, input_lengths, target_batch, target_lengths


In [ ]:
SOS_token = 2
EOS_token = 3

In [ ]:
class  EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_size, hidden_size, padding_idx=1)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        # Note: we run this all at once (over multiple batches of multiple sequences)
        embedded = self.embedding(input_seqs)
        #packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(embedded, hidden)
        #outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden

In [ ]:
class EncoderRNN_mine(nn.Module):
    
    def __init__(self, vocab_size, hidden_size, n_layers = 1, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=1)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout, bidirectional=True)
        self.hidden_size = hidden_size
    
    def forward(self, word_input, batch_size, hidden=None):
        seq_len = len(word_input)
        
        embedded = self.embedding(word_input)
        outputs, hidden = self.gru(embedded, hidden)
        encoder_out = outputs[:,:,:self.hidden_size] + outputs[:,:,self.hidden_size:]
        return encoder_out, hidden

In [ ]:
class LuongAttnDecoder(nn.Module):
    
    def __init__(self, output_size, embedding_size, hidden_size, dropout_p = 0.1, n_layers=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, embedding_size, padding_idx=1)
        self.embedding_size = embedding_size
        self.dropout = nn.Dropout(p=dropout_p)
        self.attn = nn.Linear(hidden_size, hidden_size)
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers, dropout=dropout_p)
        self.n_layers = n_layers
        self.attn_combine = nn.Linear(hidden_size + embedding_size, hidden_size)
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.output_size = output_size
        
    def score(self, rnn_output, e_outputs):
        energy = self.attn(e_outputs)
        energy = energy.transpose(0,1).transpose(1,2)
        rnn_output = rnn_output.transpose(0,1)
        energy = (rnn_output @ energy).squeeze(1)
        
        return F.softmax(energy, dim=1).unsqueeze(1)
    
    def forward(self, input_seq, hidden, e_outputs, batch_size):
        embed = self.dropout(self.embedding(input_seq)).view(1, batch_size, -1)
        rnn_output, hidden = self.gru(embed, hidden)
        attn_weights = self.score(rnn_output, e_outputs)
        context = attn_weights.bmm(e_outputs.transpose(0, 1))
        concat = torch.cat((rnn_output.squeeze(0), context.squeeze(1)), 1)
        concat = F.tanh(self.concat(concat))
        output = self.out(concat)
        
        return output, hidden, attn_weights

In [ ]:
def  as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

In [ ]:
# Configure models
attn_model = 'dot'
hidden_size = 500
n_layers = 2
dropout = 0.1
batch_size = 100
batch_size = 50

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 0.5
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_epochs = 50000
epoch = 0
plot_every = 20
print_every = 100
evaluate_every = 1000

# Initialize models
encoder = EncoderRNN_mine(len(EN_TEXT.vocab), hidden_size, n_layers, dropout=dropout)
#decoder = LuongAttnDecoderRNN(attn_model, hidden_size, output_lang.n_words, n_layers, dropout=dropout)
decoder = LuongAttnDecoder(len(FR_TEXT.vocab), hidden_size, hidden_size, dropout_p=dropout, n_layers=2)

# Initialize optimizers and criterion
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
criterion = nn.CrossEntropyLoss()

# Move models to GPU
if USE_CUDA:
    encoder.cuda()
    decoder.cuda()



# Keep track of time elapsed and running averages
start = time.time()
plot_losses = []
print_loss_total = 0 # Reset every print_every
plot_loss_total = 0 # Reset every plot_every

In [ ]:
def train(input_batches, input_lengths, target_batches, target_lengths, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word

    # Run words through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)
   
    
    # Prepare input and output variables
    decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder

    max_target_length = int(max(target_lengths))
    all_decoder_outputs = Variable(torch.zeros(max_target_length, batch_size, decoder.output_size))

    # Move new Variables to CUDA
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        all_decoder_outputs = all_decoder_outputs.cuda()

    # Run through decoder one time step at a time
    for t in range(max_target_length):
        decoder_output, decoder_hidden, decoder_attn = decoder(
            decoder_input, decoder_hidden, encoder_outputs, batch_size
        )

        all_decoder_outputs[t] = decoder_output
        decoder_input = target_batches[t] # Next input is current target

    # Loss calculation and backpropagation
    #loss = masked_cross_entropy(
     #   all_decoder_outputs.transpose(0, 1).contiguous(), # -> batch x seq
      #  target_batches.transpose(0, 1).contiguous(), # -> batch x seq
       # target_lengths
    #)
    loss = F.cross_entropy(all_decoder_outputs.view(-1, all_decoder_outputs.size(2)),
                           target_batches.view(-1), ignore_index=1)
    loss.backward()
    
    # Clip gradient norms
    ec = torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    dc = torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)

    # Update parameters with optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0], ec, dc

In [ ]:
import time

In [ ]:
start = time.time()
n_epochs = 150000
# Begin!# Begin!
ecs = []
dcs = []
eca = 0
dca = 0

while epoch < n_epochs:
    epoch += 1
    
    # Get training data for this cycle
    input_batches, input_lengths, target_batches, target_lengths = random_batch(batch_size)

    # Run the train function
    loss, ec, dc = train(
        input_batches, input_lengths, target_batches, target_lengths,
        encoder, decoder,
        encoder_optimizer, decoder_optimizer, criterion
    )
    
    # Keep track of loss
    print_loss_total += loss
    plot_loss_total += loss
    eca += ec
    dca += dc
    
   

    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg)
        print(print_summary)
        

In [ ]:
torch.save(encoder.state_dict(), "./data/encoder_att_best_20m")
torch.save(decoder.state_dict(), "./data/decoder_att_best_20m")

In [ ]:
def translate(sentence, translation):
        input_sentence = ' '.join([EN_TEXT.vocab.itos[ix] for ix in sentence.transpose(0, 1).squeeze(0).data])
        translation = ' '.join([FR_TEXT.vocab.itos[ix] for ix in translation.transpose(0, 1).squeeze(0).data])
        print('+ ' + input_sentence + '\n= ' + translation)
       
        output_sentence = []
         
        encoder_outputs, hidden = encoder(sentence, input_lengths)
    
        d_input = Variable(torch.LongTensor([SOS_token]).view(1,1).cuda())
        d_hidden = hidden[:decoder.n_layers]
        
        
    
        for i in range(30):
            d_output, d_hidden, attn_weights = decoder(d_input, d_hidden, encoder_outputs, 1)
        
            val, ix = d_output.data.topk(1)
           
            if ix[0][0] == EOS_token:
                    break
            
            output_sentence.append(FR_TEXT.vocab.itos[ix[0][0]])
            d_input = Variable(torch.LongTensor([ix[0][0]]).cuda())
            
        print('> ' + ' '.join(output_sentence) + '\n')
            
    

In [ ]:
eval_md = BucketIterator(v, batch_size=1, sort_key=lambda x: len(x.French), shuffle=True)


for i in range(20):
    
    example = next(iter(eval_md))
    translate(example.English[1:], example.French[1:])
    